In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# 데이터셋 경로와 이미지 전처리 설정

image_size = 224  # 모델에 맞는 이미지 크기로 조정
batch_size = 32

# 데이터 전처리 및 로드
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root="../data/NIH_sampled/train/Fibrosis/", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = ImageFolder(root="../data/NIH_sampled/test/Fibrosis/", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 디바이스 설정 (GPU를 사용하려면 "cuda"로 설정)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
# 학습 함수
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# 평가 함수
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()

            running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    accuracy = correct_predictions / len(dataloader.dataset)
    return epoch_loss, accuracy

In [6]:
test_accuracy = 0
while test_accuracy < 0.8:
    # MobileNetV2 모델 불러오기
    model = models.mobilenet_v2(pretrained=True)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, 2)  # 이진 분류를 위해 출력 뉴런 수를 2로 설정

    # 손실 함수 및 옵티마이저 정의
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model = model.to(device)

    # 학습 및 평가
    num_epochs = 2

    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer)
        test_loss, test_accuracy = evaluate(model, test_loader, criterion)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Test Loss: {test_loss:.4f}")
        print(f"Test Accuracy: {test_accuracy:.4f}")
        print("--------------------")

        if test_accuracy >= 0.8:
            break

Epoch 1/2
Train Loss: 0.5529
Test Loss: 0.4710
Test Accuracy: 0.7100
--------------------
Epoch 2/2
Train Loss: 0.4319
Test Loss: 0.5230
Test Accuracy: 0.7100
--------------------
Epoch 1/2
Train Loss: 0.5506
Test Loss: 1.2151
Test Accuracy: 0.6000
--------------------
Epoch 2/2
Train Loss: 0.4401
Test Loss: 0.4027
Test Accuracy: 0.8150
--------------------


In [ ]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 예측 함수
def predict(model, dataloader):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_predictions, all_labels

# 테스트 데이터셋에 대한 예측 수행
predictions, labels = predict(model, test_loader)

# Confusion Matrix 계산
cm = confusion_matrix(labels, predictions)

print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[65 35]
 [15 85]]


In [7]:
print(model.classifier[1].weight[0][:10])
print(model.classifier[1].bias[:10])

tensor([-0.0229,  0.0066, -0.0202, -0.0220,  0.0054, -0.0131, -0.0184,  0.0136,
         0.0246,  0.0046], grad_fn=<SliceBackward0>)
tensor([-0.0172, -0.0092], grad_fn=<SliceBackward0>)


In [8]:
torch.save(model, 'Fibrosis.h5')
# 정확도 : 0.815